In [52]:
import sys
import os
import json
import numbers
import time
from hashlib import sha256 as H
import random
from random import choice
import nacl.signing
import nacl.encoding
from nacl.public import PrivateKey

In [53]:
MAX_TRANSACTION_THRESHOLD = 100
MAX_INPUT_LIMIT = 1e6

In [54]:
class Transaction:
    def __init__(self, number, inp, out, sig):
        self.number = number
        self.inp = inp
        self.out = out
        self.sig = sig

def serializeTransaction(transaction):
    toSerialize = [transaction.number, transaction.inp, transaction.out, transaction.sig]
    wrapped = {'data': toSerialize}
    data = json.dumps(wrapped)
    return data
    
def serializeBlock(block):
    toSerialize = [block.tx, block.prev]
    wrapped = {'data': toSerialize}
    data = json.dumps(wrapped)
    return data
     
def deserializeBlock(block):
    wrapped = json.loads(block)
    wrapped = wrapped['data']
    return Block(wrapped[0], wrapped[1])
        
def deserializeTransaction(tx):
    wrapped = json.loads(tx)
    wrapped = wrapped['data']
    return Transaction(wrapped[0], wrapped[1], wrapped[2], wrapped[3])

def verifyWithPublicKey(pubkey, message, signature):
    try:
        verifyKey = str.encode(pubkey)
        verifyKey = nacl.signing.VerifyKey(verifyKey,
                                           encoder=nacl.encoding.HexEncoder)
        sig = str.encode(signature)
        bytesMessage = str.encode(str.encode(message).hex())
        verifyKey.verify(bytesMessage, sig, encoder=nacl.encoding.HexEncoder)
    except:
        return False
    return True

def split_money(target, nodeLimit):
    numberNodes = random.randint(1, nodeLimit)
    remaining = target
    result = []
    while remaining > 0 and len(result) < nodeLimit-1:
        remove = random.randint(1, remaining)
        result.append(remove)
        remaining -= remove
    if remaining > 0:
        result.append(remaining)
    return result

def generateKeys(totalNodes):
    pubkeys, prikeys = [], []
    for i in range(totalNodes):
        prikey = nacl.signing.SigningKey.generate()
        # Obtain the verify key for a given signing key
        pubkey = prikey.verify_key
        # Serialize the verify key to send it to a third party
        pubkey = pubkey.encode(encoder=nacl.encoding.HexEncoder)
        pubkeys.append(str(pubkey, 'utf-8'))
        prikeys.append(prikey)
    pubkeyMap = {}
    for i in range(totalNodes):
        pubkeyMap[pubkeys[i]] = i
    return pubkeys, prikeys, pubkeyMap

def createGenesisTransaction(transactionList, result, pubkeys):
    inp, out = [], []
    for pubkey in pubkeys:
        coins = random.randint(1,100)
        out.append({"value": coins, "pubkey": pubkey})
    sig = H(str.encode(str(inp) + str(out))).hexdigest()
    number = H(str.encode(str(inp) + str(out) + sig)).hexdigest()
    transaction = Transaction(number, inp, out, sig)
    transactionList.append(transaction)
    transactionAsJSON = {'number': number, 'input': inp, 'output': out, 'sig': sig}
    result.append(transactionAsJSON)

In [55]:
def generateTransactions(transactionList, result, 
                         totalNodes, totalTransactions,
                         pubkeys, prikeys, pubkeyMap):
    for i in range(1, totalTransactions):
        inputSum = 0
        transaction = transactionList[len(transactionList)-1]
        receiver = random.randint(0, len(transaction.out)-1)
        pubkeyReceiver = transaction.out[receiver]['pubkey']
        receiverInput, receiverOutput = [], []
        for receiverTransaction in transactionList:
            if (len(receiverInput) == MAX_TRANSACTION_THRESHOLD
                or inputSum >= MAX_INPUT_LIMIT):
                break
            elif receiverTransaction == transaction:
                continue
            for out in receiverTransaction.out:
                if (len(receiverInput) == MAX_TRANSACTION_THRESHOLD
                    or inputSum >= MAX_INPUT_LIMIT):
                    break
                if out['pubkey'] == pubkeyReceiver:
                    choose = choice([True, False])
                    if choose:
                        receiverInput.append({"number": receiverTransaction.number, 
                                              "output": {"value": out['value'], "pubkey": pubkeyReceiver}})
                        inputSum += out['value']
        if not receiverInput:
            receiverInput.append({"number": transaction.number, 
                                  "output": {"value": transaction.out[receiver]['value'], "pubkey": pubkeyReceiver}})
            inputSum += transaction.out[receiver]['value']
        x = split_money(inputSum, totalNodes)
        ids = random.sample(range(totalNodes), len(x))
        for j in range(len(x)):
            receiverOutput.append({"value": x[j], "pubkey": pubkeys[ids[j]]})
        serializedInput = "".join([str(inp['number']) + str(inp['output']['value']) + str(inp['output']['pubkey'])
                                  for inp in receiverInput])
        serializedOutput = "".join([str(out['value']) + str(out['pubkey']) for out in receiverOutput])
        message = str.encode(serializedInput + serializedOutput)
        prikey = prikeys[pubkeyMap[pubkeyReceiver]]
        signed = prikey.sign(message, encoder=nacl.encoding.HexEncoder)
        sig = str(signed.signature, 'utf-8')
        number = H(str.encode(serializedInput + serializedOutput + sig)).hexdigest()
        nextTransaction = Transaction(number, receiverInput, receiverOutput, sig)
        transactionList.append(nextTransaction)
        transactionAsJSON = {'number': number, 'input': receiverInput, 'output': receiverOutput, 'sig': sig}
        result.append(transactionAsJSON)

In [57]:
def run(totalCommunities, nodesPerCommunity, transactionLimitPerCommunity):
    result = []
    for i in range(totalCommunities):
        pubkeys, prikeys, pubkeyMap = generateKeys(nodesPerCommunity)
        transactionList, current = [], []
        createGenesisTransaction(transactionList, current, pubkeys)
        ntrans = random.randint(1, transactionLimitPerCommunity)
        generateTransactions(transactionList, current, 
                             nodesPerCommunity, ntrans,
                             pubkeys, prikeys, pubkeyMap)
        keys = []
        for i in range(len(pubkeys)):
            prikey = prikeys[i].encode(encoder=nacl.encoding.HexEncoder).decode()
            keys.append([pubkeys[i], prikey])
        community = {'pool': current, 'signingKeys': keys}
        result.append(community)
    return result

In [60]:
totalCommunities = 10
nodesPerCommunity = 20
transactionLimitPerCommunity = 10000
start = time.time()
result = run(totalCommunities, nodesPerCommunity, transactionLimitPerCommunity)
end = time.time()
filename = ("input/communities_" + str(totalCommunities) + "_nodes_" + 
            str(nodesPerCommunity) + "_transactions_" + str(transactionLimitPerCommunity) + ".txt")
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, 'w') as outfile:
    json.dump(result, outfile, sort_keys=False, indent=4, ensure_ascii=False)
    outfile.close()
print("Transaction file generated (contains double spends): " + filename)
print("time to generate inputs: " + str(end-start) + "s")

Transaction file generated (contains double spends): input/communities_10_nodes_20_transactions_10000.txt
time to generate inputs: 77.79522895812988s
